In [12]:
# initiating some packages
import cv2
import numpy as np
import pandas as pd
from skimage.filters import threshold_otsu
import os
import bigfish
import bigfish.stack as stack
import bigfish.detection as detection
import bigfish.multistack as multistack
import bigfish.plot as plot
import matplotlib.pyplot as plt
import tifffile as tiff
import datetime
from scipy import ndimage
from skimage.morphology import skeletonize
def current_time():
#get the current date and time
    now=datetime.datetime.now()
#format date as a string
    date_str=now.strftime('%Y-%m-%d')
    return date_str
# Set the path to the main directory containing the subdirectories with the images
path = r'F:\TRPV1_12072023'
current_time=current_time()
results_path = r"F:\TRPV1_12072023\trpv1_sub25.csv"
if not os.path.exists(results_path):
    
# Create an empty DataFrame with the desired column names
#column = ['Name', 'Amount', 'pgp_area', 'pgp_int', 'trpv1_int', 'total_dots', 'trpv1_area', 'trpv1_percent_pgp', 'Threshold']
    column = [  'Name', 
                'Amount', 
                'pgp_area', # default unit is pixel, 
                'total_dots',
                'Threshold',
               
             ]
    results_df = pd.DataFrame(columns=column)
# Save the empty DataFrame to the results file with headers
    results_df.to_csv(results_path, index=False)
# mean fluorescence intensity 
def get_intensity(image,mask):
    masking=mask>0
    # same with RawIntD in ImageJ, masked image is RGB image then to sum up will get integrated intensity
    masked=image*masking 
    area_micro = np.sum(masking)* (pixel_convert_micro *pixel_convert_micro)
    #np.sum(masked) : the integrated intensity .
    #np.sum(masked)/np.sum(masking) :the mean fluorescence intensity .
    return np.sum(masked)/np.sum(masking),np.sum(masked),area_micro # np.sum(masked) stand for integrated intensity
def count_dots_and_area(trpv1_image,mask_image,threshold):
    # define the structure for dilation
    structure = np.ones((0,0),np.uint8)
    # perform binary dilation
    dilated_mask = cv2.dilate(mask_image, structure, iterations = 1)
    # transfer the mask into uint8 type
    dilated_mask = dilated_mask.astype('uint8')
    masked_binary_image=trpv1_image*dilated_mask
    # set the parameter including threshold,log_kernel_size,and minimum_distance
    spots, threshold = detection.detect_spots(
    images=masked_binary_image, 
    return_threshold=True, 
    threshold=threshold,
    log_kernel_size=1.0,
    minimum_distance=9)
    n_dots=len(spots)
    
    return n_dots, masked_binary_image
# Loop through each subdirectory in the main directory
for dirpath, dirnames, filenames in os.walk(path):
   
    # Check if the current subdirectory contains the PGP, TRPV1, and mask subdirectories
    if 'PGP' in dirnames and 'TRPV1' in dirnames and 'PGP_pred' in dirnames:
        sample_name = os.path.basename(dirpath)
        
        pixel_convert_micro = 0.16
        # please set the threshold you want,default threshold is 30.
        for threshold in range(200, 201):
    
            # Get the corresponding filenames from the PGP, TRPV1, and mask subdirectories
            pgp_filenames = [f for f in os.listdir(os.path.join(dirpath, 'PGP')) if f.endswith('.tif')]
            trpv1_filenames = [f for f in os.listdir(os.path.join(dirpath, 'TRPV1')) if f.endswith('.tif')]
            mask_filenames = [f for f in os.listdir(os.path.join(dirpath,"PGP_pred" ,'masks')) if f.endswith('.png')]

            total_pgp_area=0
            total_pgp_int=0
            total_trpv1_int=0
            time=0
            total_dots=0
            total_trpv1_area=0
            total_trpv1_int_spe=0
            total_integrate_pgp_int= 0
            total_integrate_trpv1_int= 0
            total_integrate_trpv1_int_spe=0
            # Loop through each corresponding filename and read the images
            for pgp_filename, trpv1_filename, mask_filename, epi_filename, sub_filename in zip(pgp_filenames, trpv1_filenames, mask_filenames, epi_filenames, sub_filenames):
                #pgp,trpv1,mask path
                pgp_path = os.path.join(dirpath, 'PGP', pgp_filename)    
                trpv1_path = os.path.join(dirpath, 'TRPV1', trpv1_filename)
                mask_path = os.path.join(dirpath, "PGP_pred" ,'masks', mask_filename)
                # sample number
                filename = os.path.basename(pgp_path)
                pre_prefix = os.path.splitext(filename)[0]
                prefix=pre_prefix[-7:]

                # read pgp,trpv1,mask images
                pgp_image= plt.imread(pgp_path)
                trpv1_image= plt.imread(trpv1_path)
                mask_image=plt.imread(mask_path)

    
                # function def get_intensity to measure the integrated intensity 
                integrate_pgp_int= get_intensity(pgp_image,mask_image) [1]# mean intensity of PGP on PGP-mask
                integrate_trpv1_int= get_intensity(trpv1_image,mask_image)[1]  # mean intensity of TRPV1 on PGP-mask
                
               
                # measure the total size area of mask
                #pixel_convert_micro = 0.16 # microns depend on what the image scale is 
                pgp_area=get_intensity(pgp_image,mask_image)[2]
     
                
                time= time+ 1

                # measurement the amount of TRPV1 dots

                dots = count_dots_and_area(trpv1_image, mask_image,threshold)[0]
                pgp_int=(integrate_pgp_int * pixel_convert_micro**2)  / pgp_area  #sum of intensity per micrometer^2
                trpv1_int= (integrate_trpv1_int * pixel_convert_micro**2)/ pgp_area
            
                # read the existing csv.file
                results_df=pd.read_csv(results_path)
                # create new data to existing file
                results_dict = { 'Name':[sample_name], 
                            'Amount': [time], 
                            'pgp_area':[pgp_area],
                            'total_dots':[dots],
                            'Threshold':[threshold],
                       
                            }
                results_df = pd.DataFrame.from_dict(results_dict)
                results_df.to_csv( results_path, mode='a', index=False, header=False)

                print(results_df)


           
            

     Name  Amount  pgp_area  total_dots  Threshold
0  100020       1  212.0192           1        200
     Name  Amount  pgp_area  total_dots  Threshold
0  100020       2  132.5056           2        200
     Name  Amount  pgp_area  total_dots  Threshold
0  100020       3   126.592           0        200
     Name  Amount  pgp_area  total_dots  Threshold
0  100020       4   204.928           2        200
     Name  Amount  pgp_area  total_dots  Threshold
0  100020       5  152.7296           3        200
     Name  Amount  pgp_area  total_dots  Threshold
0  100221       1  236.9024          37        200
     Name  Amount  pgp_area  total_dots  Threshold
0  100221       2  269.2352          29        200
     Name  Amount  pgp_area  total_dots  Threshold
0  100221       3  467.9936          34        200
     Name  Amount  pgp_area  total_dots  Threshold
0  100221       4   196.992          15        200
     Name  Amount  pgp_area  total_dots  Threshold
0  100221       5  293.7856    

    Name  Amount  pgp_area  total_dots  Threshold
0  12821       3  857.5488         229        200
    Name  Amount  pgp_area  total_dots  Threshold
0  12821       4  851.6864         269        200
    Name  Amount  pgp_area  total_dots  Threshold
0  12821       5   986.112         276        200
    Name  Amount  pgp_area  total_dots  Threshold
0  13421       1  161.6384           0        200
    Name  Amount  pgp_area  total_dots  Threshold
0  13421       2  451.8656           1        200
    Name  Amount  pgp_area  total_dots  Threshold
0  13421       3  243.8656           0        200
    Name  Amount  pgp_area  total_dots  Threshold
0  13421       4  698.9568           1        200
    Name  Amount  pgp_area  total_dots  Threshold
0  13421       5  618.8032           3        200
    Name  Amount  pgp_area  total_dots  Threshold
0  14921       1  266.0096          87        200
    Name  Amount  pgp_area  total_dots  Threshold
0  14921       2  290.6112          86        200


    Name  Amount  pgp_area  total_dots  Threshold
0  47421       2   69.2992           1        200
    Name  Amount  pgp_area  total_dots  Threshold
0  47421       3   162.432           7        200
    Name  Amount  pgp_area  total_dots  Threshold
0  47421       4  248.3968          15        200
    Name  Amount  pgp_area  total_dots  Threshold
0  49321       1    466.56          45        200
    Name  Amount  pgp_area  total_dots  Threshold
0  49321       2  186.4448          12        200
    Name  Amount  pgp_area  total_dots  Threshold
0  49321       3  321.5104          25        200
    Name  Amount  pgp_area  total_dots  Threshold
0  49321       4  194.4832           8        200
    Name  Amount  pgp_area  total_dots  Threshold
0  49321       5  135.2192           8        200
    Name  Amount   pgp_area  total_dots  Threshold
0  49721       1  1140.4544         278        200
    Name  Amount  pgp_area  total_dots  Threshold
0  49721       2  1316.736         358        20

    Name  Amount  pgp_area  total_dots  Threshold
0  74421       4    183.04          16        200
    Name  Amount  pgp_area  total_dots  Threshold
0  74421       5   65.6896          16        200
    Name  Amount  pgp_area  total_dots  Threshold
0  74920       1   201.088          23        200
    Name  Amount  pgp_area  total_dots  Threshold
0  74920       2  403.3024          31        200
    Name  Amount  pgp_area  total_dots  Threshold
0  74920       3  344.7296           9        200
    Name  Amount  pgp_area  total_dots  Threshold
0  74920       4  234.9056          18        200
    Name  Amount  pgp_area  total_dots  Threshold
0  74920       5  190.0032          28        200
   Name  Amount  pgp_area  total_dots  Threshold
0  7621       1  932.9664          14        200
   Name  Amount  pgp_area  total_dots  Threshold
0  7621       2  297.9328          10        200
   Name  Amount  pgp_area  total_dots  Threshold
0  7621       3  469.5552           4        200
   Nam

C:\Users\Yuying\AppData\Local\Temp\ipykernel_18536\284359725.py:49: RuntimeWarning: invalid value encountered in divide
  return np.sum(masked)/np.sum(masking),np.sum(masked),area_micro # np.sum(masked) stand for integrated intensity
C:\Users\Yuying\AppData\Local\Temp\ipykernel_18536\284359725.py:126: RuntimeWarning: invalid value encountered in double_scalars
  pgp_int=(integrate_pgp_int * pixel_convert_micro**2)  / pgp_area  #sum of intensity per micrometer^2
C:\Users\Yuying\AppData\Local\Temp\ipykernel_18536\284359725.py:127: RuntimeWarning: invalid value encountered in double_scalars
  trpv1_int= (integrate_trpv1_int * pixel_convert_micro**2)/ pgp_area


    Name  Amount  pgp_area  total_dots  Threshold
0  89320       3       0.0           0        200
    Name  Amount  pgp_area  total_dots  Threshold
0  89320       4    0.4608           0        200


C:\Users\Yuying\AppData\Local\Temp\ipykernel_18536\284359725.py:49: RuntimeWarning: invalid value encountered in divide
  return np.sum(masked)/np.sum(masking),np.sum(masked),area_micro # np.sum(masked) stand for integrated intensity
C:\Users\Yuying\AppData\Local\Temp\ipykernel_18536\284359725.py:126: RuntimeWarning: invalid value encountered in double_scalars
  pgp_int=(integrate_pgp_int * pixel_convert_micro**2)  / pgp_area  #sum of intensity per micrometer^2
C:\Users\Yuying\AppData\Local\Temp\ipykernel_18536\284359725.py:127: RuntimeWarning: invalid value encountered in double_scalars
  trpv1_int= (integrate_trpv1_int * pixel_convert_micro**2)/ pgp_area


    Name  Amount  pgp_area  total_dots  Threshold
0  89320       5       0.0           0        200
    Name  Amount  pgp_area  total_dots  Threshold
0  89321       1    43.392           8        200
    Name  Amount  pgp_area  total_dots  Threshold
0  89321       2   93.9264           9        200
    Name  Amount  pgp_area  total_dots  Threshold
0  89321       3  122.9312          36        200
    Name  Amount  pgp_area  total_dots  Threshold
0  89321       4  171.4432          51        200
    Name  Amount  pgp_area  total_dots  Threshold
0  89321       5   86.8096          16        200
    Name  Amount  pgp_area  total_dots  Threshold
0  89521       1  112.0256           0        200
    Name  Amount  pgp_area  total_dots  Threshold
0  89521       2  119.6288           1        200
    Name  Amount  pgp_area  total_dots  Threshold
0  89521       3   69.6832           0        200
    Name  Amount  pgp_area  total_dots  Threshold
0  89521       4  120.6272          14        200


C:\Users\Yuying\AppData\Local\Temp\ipykernel_18536\284359725.py:49: RuntimeWarning: invalid value encountered in divide
  return np.sum(masked)/np.sum(masking),np.sum(masked),area_micro # np.sum(masked) stand for integrated intensity
C:\Users\Yuying\AppData\Local\Temp\ipykernel_18536\284359725.py:126: RuntimeWarning: invalid value encountered in double_scalars
  pgp_int=(integrate_pgp_int * pixel_convert_micro**2)  / pgp_area  #sum of intensity per micrometer^2
C:\Users\Yuying\AppData\Local\Temp\ipykernel_18536\284359725.py:127: RuntimeWarning: invalid value encountered in double_scalars
  trpv1_int= (integrate_trpv1_int * pixel_convert_micro**2)/ pgp_area


  Name  Amount  pgp_area  total_dots  Threshold
0  921       3       0.0           0        200
  Name  Amount  pgp_area  total_dots  Threshold
0  921       4     9.088           0        200
  Name  Amount  pgp_area  total_dots  Threshold
0  921       5   15.4624           0        200
    Name  Amount  pgp_area  total_dots  Threshold
0  92620       1   392.192           1        200
    Name  Amount  pgp_area  total_dots  Threshold
0  92620       2   248.448          25        200
    Name  Amount  pgp_area  total_dots  Threshold
0  92620       3  165.8624           2        200
    Name  Amount  pgp_area  total_dots  Threshold
0  92620       4   175.616          10        200
    Name  Amount  pgp_area  total_dots  Threshold
0  92620       5   64.2048           2        200
    Name  Amount  pgp_area  total_dots  Threshold
0  93620       1   260.736          21        200
    Name  Amount  pgp_area  total_dots  Threshold
0  93620       2  122.8288           2        200
    Name  Am